In [1]:
#importing the necessary packages 
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd
import json
import re
import os
from string import punctuation
from datetime import datetime

In [2]:
#importing the json file and previewing the first entry
jeopardy = open("jeopardy.json")
jd =json.load(jeopardy)
jd[:1]

[{'category': 'HISTORY',
  'air_date': '2004-12-31',
  'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
  'value': '$200',
  'answer': 'Copernicus',
  'round': 'Jeopardy!',
  'show_number': '4680'}]

In [4]:
#creating lists for each key in the dictionary 
category = [d['category'] for d in jd]
air_date = [d['air_date'] for d in jd]
question = [d['question'] for d in jd]
value = [d['value'] for d in jd]
answer = [d['answer'] for d in jd]
jround = [d['round'] for d in jd]
show_number = [d['show_number'] for d in jd]

In [5]:
#replacing the none value with a space so we can iterate over the list
new_value = [str(i or '') for i in value]

In [6]:
#assigning values of easy <1000 and hard >=1000
easy_hard = [0 if re.search(r'\$\d{3}$',x) else 1  for x in new_value]

In [10]:
#zipping the new hard and easy and previewing to make sure it worked correctly
list(zip(new_value, easy_hard))[:30]

[('$200', 0),
 ('$200', 0),
 ('$200', 0),
 ('$200', 0),
 ('$200', 0),
 ('$200', 0),
 ('$400', 0),
 ('$400', 0),
 ('$400', 0),
 ('$400', 0),
 ('$400', 0),
 ('$400', 0),
 ('$600', 0),
 ('$600', 0),
 ('$600', 0),
 ('$600', 0),
 ('$600', 0),
 ('$600', 0),
 ('$800', 0),
 ('$800', 0),
 ('$800', 0),
 ('$800', 0),
 ('$2,000', 1),
 ('$800', 0),
 ('$1000', 1),
 ('$1000', 1),
 ('$1000', 1),
 ('$1000', 1),
 ('$1000', 1),
 ('$400', 0)]

In [12]:
#creating a pandas dataframe
df = pd.DataFrame ({'category': category,
                    'air date': air_date,
                    'question': question,
                    'value': new_value,
                    'answer': answer,
                    'round': jround,
                    'show number': show_number,
                    'prediction': easy_hard
                   })

In [13]:
#assigning the train and test data
X_train, X_test, y_train, y_test = train_test_split(df.value, df.prediction, 
                                                    random_state = 1)

In [14]:
#Creating the vectors
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_tf = tfidf_vectorizer.fit_transform(X_train)
X_test_tf = tfidf_vectorizer.transform(X_test)

In [15]:
#performing the baysian analysis
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
predictions = naive_bayes.predict(X_test_tf)

In [16]:
#viewing the accuracy of the analysis
print('Accuracy: ', accuracy_score(y_test, predictions))

Accuracy:  0.9679899692069405
